# Cyclic Polling System with Gated Service

This example demonstrates a cyclic polling system with gated service discipline and exponential switchover times. In gated service, the server serves only jobs that were present when it arrived at the queue.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.solvers.jmt.*
import jline.util.matrix.*

In [ ]:
val model = Network("M[2]/M[2]/1-Gated")

// Block 1: nodes
val source = Source(model, "mySource")
val queue = Queue(model, "myQueue", SchedStrategy.POLLING)
val sink = Sink(model, "mySink")

In [ ]:
// Block 2: classes with different traffic intensities
val oclass1 = OpenClass(model, "myClass1")
source.setArrival(oclass1, Exp(1.0))  // λ₁ = 1.0
queue.setService(oclass1, Exp(4.0))   // μ₁ = 4.0, so E[S₁] = 0.25

val oclass2 = OpenClass(model, "myClass2")
source.setArrival(oclass2, Exp(0.8))  // λ₂ = 0.8
queue.setService(oclass2, Exp(1.5))   // μ₂ = 1.5, so E[S₂] ≈ 0.667

// Configure gated polling system with switchover times
queue.setPollingType(PollingType.GATED)
queue.setSwitchover(oclass1, Exp(1.0))  // Switchover to class 1: Exp(1.0)
queue.setSwitchover(oclass2, Exp(0.5))  // Switchover to class 2: Exp(0.5)

In [ ]:
// Block 3: topology
val P = model.initRoutingMatrix()
P.addRoute(oclass1, source, queue, sink)
P.addRoute(oclass2, source, queue, sink)
model.link(P)

In [ ]:
// Calculate theoretical utilizations and system parameters
val lambda1 = 1.0
val lambda2 = 0.8
val mu1 = 4.0
val mu2 = 1.5
val switchover1Mean = 1.0  // Mean switchover time to class 1
val switchover2Mean = 2.0  // Mean switchover time to class 2 (1/0.5)

val rho1 = lambda1 / mu1  // ρ₁ = 1.0 / 4.0 = 0.25
val rho2 = lambda2 / mu2  // ρ₂ = 0.8 / 1.5 ≈ 0.533
val rhoTotal = rho1 + rho2
val switchoverLoad = (switchover1Mean + switchover2Mean) * (lambda1 + lambda2) / 2.0

println("=== Gated Polling System Analysis ===")
println("Class 1: λ₁ = $lambda1, μ₁ = $mu1, ρ₁ = $rho1")
println("Class 2: λ₂ = $lambda2, μ₂ = $mu2, ρ₂ = $rho2")
println("Total service utilization: ρ = $rhoTotal")
println("Switchover times: E[S₁→₂] = $switchover2Mean, E[S₂→₁] = $switchover1Mean")
println("Approximate switchover load: $switchoverLoad")
println("System is ${if (rhoTotal + switchoverLoad < 1.0) "stable" else "potentially unstable"}")

In [ ]:
// Solve with MVA (solution is exact for gated polling in some cases)
try {
    val mvaSolver = MVA(model)
    val avgTableMVA = mvaSolver.avgTable
    println("\nMVA Results:")
    avgTableMVA.print()
} catch (e: Exception) {
    println("\nMVA failed for this polling model: ${e.message}")
    println("This is expected as MVA may not support all polling configurations.")
}

In [ ]:
// Try solving with JMT
try {
    val jmtOptions = JMT.defaultOptions()
    jmtOptions.samples = 1000000  // High precision for gated polling
    
    val jmtSolver = JMT(model, jmtOptions)
    val avgTableJMT = jmtSolver.avgTable
    println("\nJMT Results:")
    avgTableJMT.print()
} catch (e: Exception) {
    println("\nJMT failed for this polling model: ${e.message}")
    println("This is expected as JMT may not support all polling configurations.")
}

In [ ]:
// Display detailed system characteristics
println("\n=== Gated Service Discipline Analysis ===")
println("Model: ${model.name}")
println("Polling Type: Gated")
println("\nGated Service Rules:")
println("- Server serves only jobs present when it arrives at a queue")
println("- Jobs arriving during service wait for next visit")
println("- Prevents indefinite service periods (unlike exhaustive)")
println("- More predictable than exhaustive, less efficient than k-limited")
println("\nSwitchover Characteristics:")
println("- Class 1 → Class 2: Exponential with mean 2.0")
println("- Class 2 → Class 1: Exponential with mean 1.0")
println("- Asymmetric switchover times create bias in service")
println("\nTraffic Imbalance:")
println("- Class 1: Higher arrival rate, faster service (ρ₁ = 0.25)")
println("- Class 2: Lower arrival rate, slower service (ρ₂ ≈ 0.533)")
println("- Class 2 has higher individual utilization")
println("\nExpected Behavior:")
println("- Class 2 should have higher response times due to higher ρ")
println("- Gated discipline prevents starvation")
println("- Switchover times add overhead to system")

This example demonstrates:

1. **Gated Polling Discipline**: 
   - Server serves only jobs present when it arrives at each queue
   - Jobs arriving during service must wait for the next polling cycle
   - Provides bounded service times per visit
2. **Asymmetric System Design**:
   - Different arrival rates (1.0 vs 0.8)
   - Different service rates (4.0 vs 1.5)
   - Different switchover times (1.0 vs 0.5 rate parameters)
3. **Load Analysis**:
   - Class 1: Lower utilization but higher arrival rate
   - Class 2: Higher utilization, becomes potential bottleneck
   - Switchover overhead affects overall system performance
4. **Practical Applications**:
   - Token ring protocols
   - Time-division multiple access (TDMA) systems
   - Manufacturing systems with setup times
   - Database systems with transaction classes

Gated service is particularly important in communication systems where fairness and bounded response times are critical, even at the cost of some efficiency compared to exhaustive service.